In [37]:
from google.colab import drive
drive.mount('/content/drive')  # This will ask for authorization to access your Google Drive


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
import pandas as pd

# Provide the correct file path
file_path = '/content/drive/MyDrive/Rotten_Tomatoes_Movies3.xls/Rotten_Tomatoes_Movies3.xls'

# Load the dataset
data = pd.read_excel(file_path)

# Check the first few rows of the dataset
print(data.head())


                                         movie_title  \
0  Percy Jackson & the Olympians: The Lightning T...   
1                                        Please Give   
2                                                 10   
3                    12 Angry Men (Twelve Angry Men)   
4                       20,000 Leagues Under The Sea   

                                          movie_info  \
0  A teenager discovers he's the descendant of a ...   
1  Kate has a lot on her mind. There's the ethics...   
2  Blake Edwards' 10 stars Dudley Moore as George...   
3  A Puerto Rican youth is on trial for murder, a...   
4  This 1954 Disney version of Jules Verne's 20,0...   

                                   critics_consensus rating  \
0  Though it may seem like just another Harry Pot...     PG   
1  Nicole Holofcener's newest might seem slight i...      R   
2                                                NaN      R   
3  Sidney Lumet's feature debut is a superbly wri...     NR   
4  One of D

In [43]:
# Preprocess the target (audience_rating)
y = data['audience_rating'].ffill()  # Forward fill for missing values in the target
y = y.dropna()  # Drop any NaN values


In [44]:
# Select features (X)
X = data.drop(columns=['audience_rating'])  # Drop the target column

# Handle missing values in the features
X = X.fillna(method='ffill')  # Forward fill remaining missing values


<ipython-input-44-c0cba951e4a6>:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X = X.fillna(method='ffill')  # Forward fill remaining missing values


In [45]:
# Convert all object columns to strings to avoid type mismatch
for col in X.select_dtypes(include=['object']).columns:
    X[col] = X[col].astype(str)


In [46]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ensure that there are no NaN values in X_train or y_train
assert X_train.isnull().sum().sum() == 0, "X_train contains NaN values"
assert y_train.isnull().sum() == 0, "y_train contains NaN values"


In [47]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Identify numerical and categorical columns
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps in a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)


In [48]:
from sklearn.ensemble import RandomForestRegressor

# Create a pipeline with preprocessing and regressor
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])


In [50]:
# Train the model
model.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['runtime_in_minutes', 'tomatometer_rating', 'tomatometer_count'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['movie_title', 'movie_info', 'critics_consensus', 'rating', 'genre',
       'directors', 'writers', 'cast', 'studio_name', 'tomatometer_status'],
      dtype='object'))])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [51]:
# Make predictions on the test set
y_pred = model.predict(X_test)


In [52]:
from sklearn.metrics import r2_score, mean_squared_error

# Evaluate the model
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

# Print the results
print(f"R² Score: {r2}")
print(f"Mean Squared Error: {mse}")


R² Score: 0.4663378305699525
Mean Squared Error: 221.7713031550481


In [54]:
import joblib

# Save the pipeline (optional)
joblib.dump(model, '/content/drive/My Drive/audience_rating_pipeline.pkl')

print("Model training and validation completed!")


Model training and validation completed!


In [ ]:
# ===============================================
# Author: R Suriyamoorthy
# Role: Electronics and Communication Engineering Student
# Key Achievements: 2nd place in FreqFest Paper Presentation, 4th Rank in Snowflake MCQ Contest, Leader of Hackathon Event Team
# Contact: 21urec047@aaacet.ac.in
# Project: Audience Rating Prediction Model
# Date: 2024-12-20
# ===============================================
